In [1]:

import numpy as np
from absl.testing import parameterized

from tensorflow.python.keras import keras_parameterized
from tensorflow.keras.backend import floatx
import tensorflow as tf

from autopycoin.dataset.generator import WindowGenerator

from autopycoin.models import PoolNBEATS

from autopycoin.utils import layer_test, check_attributes
from autopycoin.losses import QuantileLossError
from autopycoin.layers.nbeats_layers import GenericBlock, TrendBlock, SeasonalityBlock
from autopycoin.models import (
    create_interpretable_nbeats,
    create_generic_nbeats,
    NBEATS,
    Stack,
)
from autopycoin.data import random_ts


data = random_ts(n_steps=30, n_variables=2)

w = WindowGenerator(
        input_width=6,
        label_width=3,
        shift=3,
        valid_size=10,
        test_size=0,
        flat=False,
        preprocessing=lambda x,y: (x, (x, y))
        )
w.from_array(data, input_columns=[0, 1], label_columns=[0, 1])

model = [create_generic_nbeats(
            label_width=3,
            g_forecast_neurons=2,
            g_backcast_neurons=2,
            n_neurons=2,
            n_blocks=2,
            n_stacks=2,
            drop_rate=0,
            share=True,
        ),create_generic_nbeats(
            label_width=3,
            g_forecast_neurons=2,
            g_backcast_neurons=2,
            n_neurons=2,
            n_blocks=2,
            n_stacks=2,
            drop_rate=0,
            share=True,
        )]

model = create_generic_nbeats(
            label_width=3,
            share=False
        )

model.compile(
    tf.keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam",
    ),
    loss=['mse', QuantileLossError([0.5, .7, 0.8])],
    metrics="mae",
)

for i, j in w.train.take(1):
    pass

#model(i)
# Issue 13
model.fit(w.train, epochs=10, validation_data=w.valid)

quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5, 0.7, 0.8]
quatiles_in_model [0.2, 0.3, 0.5

KeyboardInterrupt: 

In [9]:
i

<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
array([[[ 0.901755  , -6.2729874 ],
        [-0.64673245,  0.6860294 ],
        [-0.646649  ,  0.6887942 ],
        [-0.6465657 ,  0.6915594 ],
        [-0.64648294,  0.69432706],
        [-0.64639956,  0.697092  ]]], dtype=float32)>

In [8]:
type(create_generic_nbeats(
            label_width=3,
            g_forecast_neurons=2,
            g_backcast_neurons=2,
            n_neurons=2,
            n_blocks=2,
            n_stacks=2,
            drop_rate=0,
            share=True,
        ))

autopycoin.models.nbeats.NBEATS

In [1]:
from autopycoin.data import random_ts
from autopycoin.models import PoolNBEATS, create_interpretable_nbeats
from autopycoin.dataset import WindowGenerator
import tensorflow as tf
import pandas as pd

data = random_ts(n_steps=1000,
                 trend_degree=2,
                 periods=[10],
                 fourier_orders=[10],
                 trend_mean=0,
                 trend_std=1,
                 seasonality_mean=0,
                 seasonality_std=1,
                 batch_size=1,
                 n_variables=1,
                 noise=True,
                 seed=42)
data = pd.DataFrame(data[0].numpy(), columns=['test'])

w = WindowGenerator(
       input_width=70,
       label_width=10,
       shift=10,
       test_size=50,
       valid_size=10,
       flat=True,
       batch_size=32,
       preprocessing=lambda x,y: (x, (x, y))
    )

w = w.from_array(data=data,
       input_columns=['test'],
       label_columns=['test'])

model = PoolNBEATS(
            label_width=10,
            n_models=3,
            nbeats_models=create_interpretable_nbeats)
model.compile(tf.keras.optimizers.Adam(
   learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
   name='Adam'), loss=['mse', 'mse', 'mse'], metrics=['mae'])

model.build(tf.TensorShape(w.train.element_spec[0].shape))
"""
model.predict(w.test.take(1))"""

print(model.nbeats[0].trainable_variables)


[<tf.Variable 'trend_stack/trend_block/fc_kernel_trend_block_0:0' shape=(40, 252) dtype=float32, numpy=
array([[-0.07706008,  0.05801174,  0.11065568, ...,  0.14334212,
        -0.05718359, -0.08760013],
       [ 0.13892151, -0.12866658,  0.03056897, ..., -0.14270799,
        -0.11347268,  0.07306087],
       [ 0.02456225, -0.11336257,  0.1074241 , ..., -0.04218578,
        -0.00538136,  0.07833546],
       ...,
       [-0.11951011,  0.13617976,  0.0801385 , ..., -0.0824777 ,
        -0.08619782, -0.0670284 ],
       [ 0.02267867, -0.0728267 , -0.06264693, ...,  0.01911539,
         0.0314445 , -0.04968672],
       [-0.13443601, -0.03674489, -0.06764066, ..., -0.13181783,
         0.11497615, -0.11455805]], dtype=float32)>, <tf.Variable 'trend_stack/trend_block/fc_bias_trend_block_0:0' shape=(1, 252) dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.

In [2]:
model.fit(w.train, epochs=10, validation_data=w.valid)

Epoch 1/10
27/27 [==============================] - 5s 54ms/step - loss: 17.2960 - output_1_1_loss: 2.9781 - output_1_2_loss: 10.6843 - output_2_1_loss: 3.6336 - output_1_1_mae: 1.1894 - output_1_2_mae: 2.2343 - output_2_1_mae: 1.3523 - output_2_2_mae: 2.1851 - output_3_1_mae: 2.6136 - output_3_2_mae: 2.3830 - val_loss: 12.1720 - val_output_1_1_loss: 1.0048 - val_output_1_2_loss: 9.9730 - val_output_2_1_loss: 1.1942 - val_output_1_1_mae: 0.7907 - val_output_1_2_mae: 2.1082 - val_output_2_1_mae: 0.8630 - val_output_2_2_mae: 2.1118 - val_output_3_1_mae: 2.1233 - val_output_3_2_mae: 2.1324
Epoch 2/10
27/27 [==============================] - 1s 26ms/step - loss: 10.0580 - output_1_1_loss: 0.6914 - output_1_2_loss: 8.5387 - output_2_1_loss: 0.8279 - output_1_1_mae: 0.6668 - output_1_2_mae: 2.0675 - output_2_1_mae: 0.7294 - output_2_2_mae: 2.1345 - output_3_1_mae: 2.0127 - output_3_2_mae: 2.1442 - val_loss: 10.6272 - val_output_1_1_loss: 0.7855 - val_output_1_2_loss: 8.9938 - val_output_2_1_

In [14]:
from tensorflow import keras
import numpy as np
class TemporalSoftmax(keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        #print("e", tf.exp(inputs))
        print('y')
        print('r', mask)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        print(inputs_exp)
        inputs_sum = tf.reduce_sum(
            inputs_exp * broadcast_float_mask, axis=-1, keepdims=True
        )
        return inputs_exp / inputs_sum


inputs = keras.Input(shape=(None,), dtype="int32")
x = keras.layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = keras.layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)

model = keras.Model(inputs, outputs)
y = model(np.random.randint(0, 10, size=(32, 100)), np.random.random((32, 100, 1)))

y
r Tensor("Placeholder_1:0", shape=(None, None), dtype=bool)
Tensor("temporal_softmax_11/mul:0", shape=(None, None, 1), dtype=float32)
y
r tf.Tensor(
[[ True  True  True ...  True False  True]
 [ True  True  True ...  True  True  True]
 [ True False  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]], shape=(32, 100), dtype=bool)
tf.Tensor(
[[[0.9769019]
  [0.9864606]
  [1.000157 ]
  ...
  [0.947646 ]
  [0.       ]
  [0.947646 ]]

 [[1.000157 ]
  [1.057268 ]
  [1.0350581]
  ...
  [0.9683153]
  [0.947646 ]
  [0.9683153]]

 [[1.000157 ]
  [0.       ]
  [1.057268 ]
  ...
  [0.947646 ]
  [0.947646 ]
  [0.9690754]]

 ...

 [[1.000157 ]
  [1.0350581]
  [1.0350581]
  ...
  [0.9204331]
  [0.9769019]
  [1.0350581]]

 [[0.9769019]
  [1.057268 ]
  [0.9204331]
  ...
  [1.000157 ]
  [1.0350581]
  [0.9864606]]

 [[0.947646 ]
  [0.9690754]
  [0.9683153]
  ...
  [1.000157 ]
  [0.9204331]
  

In [3]:
from absl.testing import parameterized
import pandas as pd

import tensorflow as tf
from autopycoin.data.generate import random_ts
from tensorflow.python.keras import keras_parameterized

from autopycoin.models import create_interpretable_nbeats, Model
from autopycoin.dataset.generator import WindowGenerator
from autopycoin.losses import QuantileLossError

data = random_ts(
    n_steps=400,
    trend_degree=2,
    periods=[10],
    fourier_orders=[10],
    trend_mean=0,
    trend_std=1,
    seasonality_mean=0,
    seasonality_std=1,
    batch_size=1,
    n_variables=1,
    noise=True,
    seed=42,
)

data = pd.DataFrame(data[0].numpy(), columns=["test"])
data["date"] = range(400)

w = WindowGenerator(
    input_width=20,
    label_width=50,
    shift=50,
    test_size=10,
    valid_size=10,
    flat=True,
    batch_size=32,
)

w = w.from_array(
    data=data,
    input_columns=["test"],
    label_columns=["test"],
)

inputs = tf.keras.Input(shape=(20,))
output = tf.keras.layers.Dense(50)(inputs)
output = tf.expand_dims(output, axis=0)
model = Model(inputs=inputs, outputs=output)

model.compile(
    tf.keras.optimizers.Adam(
        learning_rate=0.015,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam",
    ),
    loss=QuantileLossError([0.1, 0.3, 0.5]),
    metrics=["mae"],
)

model.fit(w.train, validation_data=w.valid, epochs=1)
output = model.predict(w.train)

self.assertEqual(output.shape, (213, 50, 1))

7/7 [==============================] - 1s 53ms/step - loss: 224.3874 - mae: 2.8857 - val_loss: 55.1039 - val_mae: 2.2042


NameError: name 'self' is not defined